##### A szociológia elmúlt évtizede #####

A jstor.com akadémiai keresőoldal első 100 találatának scrapelésével szereztem meg a 2010 évek óta publikált tanulmányok egy kis részének a nevét, szerző(i)t, a publikálás évszámát, és a téma kulcsszavakat. Ennek segítségével megvizsgáltam az kulcsszavak relatív gyakoriságát, hogy ebből következtessek arra mik voltak a "hot" topicok a szociológiában a 2010-es évek óta. Továbbfejlesztési irány: évek szerinti bontás, hiv. számmal szerzők, szerzők social networkje (akadémiai szerzőpárosok, legaktívabban publikálók, legnépszerűbb folyóiratok stb..)

In [218]:
#Modules import

import json
import sys
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [263]:
#Scrapping code
#Itt technika problémák miatt kézzel módosítottam a letöltött html fájl nevét (json_1, json_2, json_3, json_4) és mentettem el külön jsonba.

next_url = "https://www.jstor.org/action/doBasicSearch?searchType=facetSearch&cty_journal_facet=am91cm5hbA%3D%3D&sd=2000&ed=&Query=sociology"
#Dic for jstor
dic_js_info = {}


while next_url:
    #Next url
    print(next_url)
    #Index
    index = 0
    #Downloaded html file open
    with open("jstor_4.html", "r", encoding = 'utf-8') as file:
    #making a beatifulsoup object
        list_of_js = BeautifulSoup(file.read(),'html5lib').find("ol", {"class": "list-searchResults mln"}).find_all("li", {"class": "row result-item"})
        
        
        #Search page for cycle
        for journals in list_of_js:
            dic_js_info[index] = {}
        
            ##Information scrapping

            #Title
            dic_js_info[index] ["Title"] = journals.find("div", {"class": "title"}).text.strip()
            #Author
            try:
                dic_js_info[index] ["Author"] = journals.find("div", {"class": "contrib"}).text.strip()
            except:
                dic_js_info[index] ["Author"] = "None"
            #Journal
            dic_js_info[index] ["Journal"] = journals.find("div", {"class": "src break-word"}).text.strip()
            #Topics
            try:
                list_of_topraw = journals.find("div", {"class": "topic-evaluation-pane mtm"}).find_all("a")
                list_of_top = []
                for i in list_of_topraw:
                    list_of_top.append(i.text.strip())

                dic_js_info[index] ['Topics'] =  list_of_top 
            except:
                dic_js_info[index] ["Topics"] = []
            
            #Next index
            index += 1
        #Save and export into json
        #Making a dataframe
        js_df = pd.DataFrame.from_dict(dic_js_info, orient = "index", columns = ["Title", "Author", "Journal", "Topics"])
        #Dataframe into json
        js_df.to_json(r'C:\Users\Ilyés Márton\Documents\Asztal_2019_11_18\2020_1\Prog\jstor_4.json')
        
    break
    
#Output
#dic_js_info

https://www.jstor.org/action/doBasicSearch?searchType=facetSearch&cty_journal_facet=am91cm5hbA%3D%3D&sd=2000&ed=&Query=sociology


In [264]:
#Database cleaning

#Merge jsons
df = pd.read_json(os.path.join("jstor_1.json")).T
df_2 = pd.read_json(os.path.join("jstor_2.json")).T
df_3 = pd.read_json(os.path.join("jstor_3.json")).T
df_4 = pd.read_json(os.path.join("jstor_4.json")).T
merged_df = pd.concat([df, df_2, df_3, df_4], axis = 1)
merged_tr=merged_tr.reset_index(drop=True)

#Transpose axis
merged_tr = merged_df.transpose(copy=True)

#Make them list
merged_tr['Journal'] = merged_tr['Journal'].str.split(',')

#Row length reconfig
pd.options.display.max_colwidth = 500

#Journal title and vol 
merged_tr['J_Title']=merged_tr.apply(lambda x: x['Journal'][0], axis=1)
merged_tr['J_Vol']=merged_tr.apply(lambda x: x['Journal'][1], axis=1)

In [265]:
merged_tr.head()

,Title,Author,Journal,Topics,J_Title,J_Vol
0,Investigating the Apathy toward Applied Sociology,Andrew C. Cohen,"[Journal of Applied Social Science, Vol. 5, No. 2 (September 2011), pp. 53-65]","[Public sociology, Undergraduate students, Questionnaires, Apathy, Social psychology, Clinical sociology, Universities]",Journal of Applied Social Science,Vol. 5
1,Career Trajectories of AB Sociology Graduates of Xavier University,MA. RHEA GRETCHEN A. ABUSO,"[Philippine Sociological Review, Vol. 66 (2018), pp. 83-102]","[Social research, Educational research, Human resources, Applied sociology, Graduates, Occupations, Research methods, Social sciences]",Philippine Sociological Review,Vol. 66 (2018)
2,Individualized Medical Sociology: Placing Sociology in Medical Practice,Costas S. Constantinou,"[Journal of Applied Social Science, Vol. 9, No. 2 (September 2015), pp. 182-190]","[Medical sociology, Medical education, Medical students, Medical practice, Social stigma, Sons, Problem based learning, Textbooks]",Journal of Applied Social Science,Vol. 9
3,Public Sociology and its Publics,Herbert J. Gans,"[The American Sociologist, Vol. 47, No. 1 (March 2016), pp. 3-11]","[Public sociology, Journalism, Social media, Social research, Websites, Electronic books, Publishing industry, Audiences]",The American Sociologist,Vol. 47
4,No Path to Paradise: Deconstructing the Promise of Public Sociology,Tony Christensen,"[The American Sociologist, Vol. 44, No. 1 (March 2013), pp. 23-41]","[Public sociology, Critical theory, Civil society, Pathology, Social issues, Political sociology, Social justice, Graduate students, Public sphere]",The American Sociologist,Vol. 44


In [267]:
#Data visualization

topics = pd.DataFrame(set(sum(merged_tr["Topics"], []))).rename(columns={0: "Topic"})
topics["Freq"] = topics["Topic"].apply(
    lambda m: merged_tr["Topics"].apply(lambda l: m in l).sum()
)

topics.sort_values(by="Freq", ascending=False).head(10)

,Topic,Freq
342,Social research,42
51,Public sociology,27
375,Political sociology,19
309,Social sciences,14
150,Sociology,13
325,Sociology of religion,12
385,Social issues,12
175,Educational sociology,10
121,College instruction,10
186,Traditions,9
